In [25]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=12

println("Number of variables: n=",n)

@polyvar x[1:n]# variables

#function to get a random quadratic polynomial of variables x(T)
function generate_random_poly(T::UnitRange{Int64})
    v=reverse(monomials(x[T],2))
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

# ball constraints on subsets of variables
u=4# clique size
p=floor(Int64,n/u) #number of cliques

# indices of variables on each clique
I=Vector{UnitRange{Int64}}(undef,p)
I[1]=1:u 
I[2:p-1]=[u*(j-1):u*j for j in 2:p-1]
I[p]=u*(p-1):n

# random quadratic objective function f
vecf=[generate_random_poly(I[j]) for j in 1:p] #vector of separable polynomials on each clique
f=sum(vecf)

# ball constraints on each clique
m=ceil(Int64, n/5)# number of the equality constraints

r=floor(Int64,m/p)
J=[(j-1)*r+1:j*r for j in 1:p-1]# assign equality constraints
append!(J,[(p-1)*r+1:m])

g=Vector{Polynomial{true,Float64}}(undef,m)


# get a random point satisfies the inequality constraints
randx=rand(Float64,n)

for j in 1:p
    randx[I[j]]=randx[I[j]]./sum(randx[I[j]])
    randx[I[j]]=rand(Float64,1)[1]*randx[I[j]]
end

for j in 1:p
    for i in J[j]
        g[i]=generate_random_poly(I[j])
        g[i]+=0.125-g[i](x => randx) #make the random point satisfy the equality constraint h[i](randx) = 0
    end
    g[J[j][1]]=1-sum(x[I[j]])
end

m=length(g)
println("Number of inequality constraints: m=",m)


l=ceil(Int64, n/5)# number of the equality constraints

r=floor(Int64,l/p)
W=[(j-1)*r+1:j*r for j in 1:p-1]# assign equality constraints
append!(W,[(p-1)*r+1:l])

h=Vector{Polynomial{true,Float64}}(undef,l)


for j in 1:p
    for i in W[j]
        h[i]=generate_random_poly(I[j])
        h[i]-=h[i](x => randx) #make the random point satisfy the equality constraint h[i](randx) = 0
    end
end

l=length(h)
println("Number of equality constraints: l=",l)


***Problem setting***
Number of variables: n=12
Number of inequality constraints: m=3
Number of equality constraints: l=3


In [26]:
# using SparseArrays

# include("../src/InterRelax.jl")
# using .InterRelax

# n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);

# k=4
# t=2
# s=10

# @time opt_val1,opt_sol=InterRelax.RelaxSparseCSTS(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,t,s,assign="min",alg="MD",minimize=true,solver="Mosek",comp_opt_sol=false);

In [53]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);
#data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
#include(data*"/sparsePOPcliq$(u)nineq$(m)neq$(l).jl")

k=3
s=6

@time opt_val1,opt_sol=InterRelax.RelaxSparse_without_multiplier(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
    lmon_f,supp_f,coe_f,dg,dh,s,k,
    assign="min",alg="MD",minimize=true,solver="Mosek",comp_opt_sol=false);

**Interrupted relaxation based on Handelman's Positivstellensatz**
Relaxation order: k=3
Sparsity order: s=6
------------------------------------------------------
The clique sizes of varibles:
[5, 4]
[2, 1]
------------------------------------------------------
  Number of cliques: p=3
  Largest clique size: u=5
 Maximal matrix size:6
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 139             
  Cones                  : 0               
  Scalar variables       : 108             
  Matrix variables       : 20              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00         

In [52]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);
#data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
#include(data*"/sparsePOPcliq$(u)nineq$(m)neq$(l).jl");

k=1
s=6
d=Int64(maximum([sum(supp_f[:,i]) for i in 1:lmon_f]))

@time opt_val1,opt_sol=InterRelax.RelaxSparse(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,s,d,assign="min",alg="MD",minimize=true,solver="Mosek",comp_opt_sol=false);

**Interrupted relaxation based on Putinar-Vasilescu's Positivstellensatz**
Relaxation order: k=1
Sparsity order: s=6
Sparsity order: d=2
------------------------------------------------------
The clique sizes of varibles:
[5, 4]
[2, 1]
------------------------------------------------------
  Number of cliques: p=3
  Largest clique size: u=5
 Maximal matrix size:6
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 198             
  Cones                  : 0               
  Scalar variables       : 165             
  Matrix variables       : 20              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 4
Eliminator terminated.
Eliminator - tries                  : 1                 time      

In [30]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);
#data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
#include(data*"/sparsePOPcliq$(u)nineq$(m)neq$(l).jl");

k=2

opt=InterRelax.TSSOS_CS(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,k);

**Semidefinite relaxation based on Putinar's Positivstellensatz**
Relaxation order: k=2
************************TSSOS************************
TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[5, 4]
[2, 1]
------------------------------------------------------
Obtained the variable cliques in 0.00013745 seconds. The maximal size of cliques is 5.
Assembling the SDP...
There are 312 affine constraints.
SDP assembling time: 0.001099991 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 312             
  Cones                  : 0               
  Scalar variables       : 58              
  Matrix variables       : 18              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker t

3   8.2e-02  1.4e-02  7.0e-04  1.18e+00   -6.079922556e-01  -6.033168737e-01  1.4e-02  0.01  
4   2.1e-02  3.4e-03  1.1e-04  8.87e-01   -5.595953754e-01  -5.585383856e-01  3.4e-03  0.01  
5   5.4e-03  9.0e-04  1.5e-05  9.43e-01   -5.560338824e-01  -5.557738861e-01  9.0e-04  0.01  
6   9.4e-04  1.6e-04  1.1e-06  9.83e-01   -5.558334353e-01  -5.557896368e-01  1.6e-04  0.02  
7   2.9e-05  4.8e-06  6.4e-09  9.93e-01   -5.558123035e-01  -5.558111123e-01  4.8e-06  0.02  
8   1.6e-06  2.7e-07  8.5e-11  1.00e+00   -5.558163832e-01  -5.558163157e-01  2.7e-07  0.02  
9   3.3e-07  5.5e-08  7.7e-12  1.01e+00   -5.558164495e-01  -5.558164358e-01  5.5e-08  0.02  
10  6.8e-09  1.2e-09  2.2e-14  1.00e+00   -5.558164200e-01  -5.558164197e-01  1.1e-09  0.03  
Optimizer terminated. Time: 0.03    

SDP solving time: 0.028016372 seconds.
optimum = -0.5558164199907546
  0.031085 seconds (59.29 k allocations: 5.720 MiB)


In [44]:
using SparseArrays

include("../src/InterRelax.jl")
using .InterRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterRelax.get_info(x,f,g,h,sparse=true);
#data="/home/hoanganh/Desktop/math-topics/InterruptedRelax/codes/dataPOP2" # path of data
#include(data*"/densePOPvar$(n)nineq$(m)neq$(l).jl");

using TSSOS

k=2
vars,pop=InterRelax.get_POP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)
@time opt,sol,data=TSSOS.cs_tssos_first(pop,vars,k,numeq=l,CS=false,TS=false,solver="Mosek");

************************TSSOS************************
TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[10]
[1]
------------------------------------------------------
Assembling the SDP...
There are 1001 affine constraints.
SDP assembling time: 0.008555836 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1001            
  Cones                  : 0               
  Scalar variables       : 133             
  Matrix variables       : 13              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00    